<a href="https://colab.research.google.com/github/lucia-galiero/TICO-19_NMT_LLM/blob/main/fuzzy_match_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
from sentence_transformers import SentenceTransformer, util
import spacy

# Load SpaCy model for tokenization
nlp = spacy.load("en_core_web_sm")

# Load SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Function to tokenize and preprocess sentences
def tokenize_and_preprocess(sentence):
    doc = nlp(sentence)
    return " ".join([token.text for token in doc if not token.is_punct and not token.is_space])

# Load data
def load_txt(file_path):
    with open(file_path, 'r', encoding='utf-8-sig') as txt_file:
        return [line.strip() for line in txt_file if line.strip()]

# Main processing function
def process_test_set(test_file, candidate_file1, candidate_file2, output_file):
    test_sentences = load_txt(test_file)
    candidate_sentences_1 = load_txt(candidate_file1)
    candidate_sentences_2 = load_txt(candidate_file2)

    # Combine candidates from both files and remove test sentences from the pool
    all_candidates = set(candidate_sentences_1 + candidate_sentences_2)
    excluded_sentences = set(test_sentences)
    candidate_sentences = [s for s in all_candidates if s not in excluded_sentences and len(s.split()) <= 80]  # Exclude sentences > 80 words

    # Create a mapping to identify the source of each candidate
    candidate_sources = {
        **{s: candidate_file1 for s in candidate_sentences_1},
        **{s: candidate_file2 for s in candidate_sentences_2}
    }

    with open(output_file, 'w', encoding='utf-8') as out_file:
        for test_sentence in test_sentences:
            # Tokenize and preprocess the test sentence
            preprocessed_test_sentence = tokenize_and_preprocess(test_sentence)

            # Tokenize and preprocess candidate sentences
            preprocessed_candidates = [tokenize_and_preprocess(s) for s in candidate_sentences]

            # Compute sentence embeddings and similarity scores
            test_embedding = model.encode(preprocessed_test_sentence, convert_to_tensor=True)
            candidate_embeddings = model.encode(preprocessed_candidates, convert_to_tensor=True)
            similarity_scores = util.pytorch_cos_sim(test_embedding, candidate_embeddings).squeeze(0).tolist()

            # Get top 2 similar sentences with a score of at least 0.60
            scored_sentences = [(candidate_sentences[i], similarity_scores[i]) for i in range(len(candidate_sentences))]
            scored_sentences = [(s, score) for s, score in scored_sentences if score > 0.00]
            scored_sentences = sorted(scored_sentences, key=lambda x: x[1], reverse=True)[:2]

            examples = []
            for sentence, score in scored_sentences:
                source_file = candidate_sources[sentence]
                examples.append((sentence, "to be added", score, source_file))

            # Print results
            print(f'Test Sentence: "{test_sentence}"')
            for i, (source, _, score, source_file) in enumerate(examples):
                print(f'Example {i+1} ({score:.2f}): "{source}" | "to be added" (from {source_file})')

            # Write to file
            out_file.write("\n")
            for source, target, _, source_file in examples:
                out_file.write(f"English: {source}\n")
                out_file.write(f"Italian: {target}\n")
            out_file.write(f"English: {test_sentence}\n")
            out_file.write(f"MT: \"to be added\"\n")
            out_file.write(f"Italian:\"to be added\"\n")
            out_file.write("\n")

# Example usage
process_test_set('test_en.txt', 'training_en.txt', 'tuning.txt', 'LLAMA_2S_prompts.txt')


Test Sentence: "Preliminary Estimates of the Prevalence of Selected Underlying Health Conditions Among Patients with Coronavirus Disease 2019 — United States, February 12–March 28, 2020"
Example 1 (0.71): "Among 122,653 U.S. COVID-19 cases reported to CDC as of March 28, 2020, 7,162 (5.8%) patients had data available pertaining to underlying health conditions or potential risk factors; among these patients, higher percentages of patients with underlying conditions were admitted to the hospital and to an ICU than patients without reported underlying conditions." | "to be added" (from tuning.txt)
Example 2 (0.70): "On March 11, 2020, the World Health Organization declared Coronavirus Disease 2019 (COVID-19) a pandemic." | "to be added" (from tuning.txt)
Test Sentence: "Reports from China and Italy suggest that risk factors for severe disease include older age and the presence of at least one of several underlying health conditions."
Example 1 (0.69): "Published reports from China and Ita

In [ ]:
import csv
import unicodedata

def normalize_string(s):
    """Normalize strings to remove encoding issues and hidden characters."""
    return unicodedata.normalize("NFKC", s).strip()

def update_output_file_in_place(output_file, tsv_file, test_set_file, test_set_translation_file):
    # Load translations from TSV
    tsv_translations = {}
    with open(tsv_file, 'r', encoding='utf-8-sig') as tsv:  # Handle UTF-8 BOM
        reader = csv.DictReader(tsv, delimiter='\t')
        for row in reader:
            source = normalize_string(row['sourceString'])
            target = normalize_string(row['targetString'])
            tsv_translations[source] = target

    # Load test set (English sentences)
    with open(test_set_file, 'r', encoding='utf-8-sig') as test_file:
        test_sentences = [normalize_string(line) for line in test_file if line.strip()]

    # Load test set translations (MT output)
    with open(test_set_translation_file, 'r', encoding='utf-8-sig') as translation_file:
        test_translations = [normalize_string(line) for line in translation_file if line.strip()]

    # Read the original file and modify it in memory
    with open(output_file, 'r', encoding='utf-8-sig') as out_file:
        lines = out_file.readlines()

    updated_lines = []
    i = 0
    while i < len(lines):
        line = normalize_string(lines[i])
        if line.startswith("English:"):
            english_sentence = normalize_string(line[len("English:"):])
            next_line = normalize_string(lines[i + 1])

            # Handle Italian translations
            if next_line.startswith("Italian: to be added"):
                if english_sentence in tsv_translations:
                    italian_translation = tsv_translations[english_sentence]
                else:
                    print(f"ERROR: Missing Italian translation for '{english_sentence}'")
                    italian_translation = ""

                updated_lines.append(f"English: {english_sentence}\n")
                updated_lines.append(f"Italian: {italian_translation}\n")
                i += 2  # Skip the "Italian: to be added" line

            # Handle MT translations
            elif next_line.startswith("MT:"):
                if english_sentence in test_sentences:
                    line_number = test_sentences.index(english_sentence)
                    mt_translation = test_translations[line_number]
                else:
                    print(f"ERROR: Missing MT translation for '{english_sentence}'")
                    mt_translation = "Translation not found"

                updated_lines.append(f"English: {english_sentence}\n")
                updated_lines.append(f"MT: {mt_translation}\n")
                updated_lines.append(f"Italian:\n")  # Leave empty for MT translations
                i += 3  # Skip MT and Italian lines
            else:
                updated_lines.append(line + "\n")
                i += 1
        else:
            updated_lines.append(line + "\n")
            i += 1

    # Overwrite the original file with the updated content
    with open(output_file, 'w', encoding='utf-8-sig') as out_file:
        out_file.writelines(updated_lines)

# Example usage
update_output_file_in_place(
    output_file='LLAMA_2S_prompts.txt',
    tsv_file='TICO19_En-It.tsv',
    test_set_file='test_en.txt',
    test_set_translation_file='baseline_llama3_2_90b.txt'
)


ERROR: Missing Italian translation for 'A comparison of SARS-CoV-2 with MERS-CoV and SARS-CoV regarding their symptoms, mortality, and R0 is presented in Table 1.'
ERROR: Missing Italian translation for 'A comparison of SARS-CoV-2 with MERS-CoV and SARS-CoV regarding their symptoms, mortality, and R0 is presented in Table 1.'
